## 1️⃣ `config.yaml` → Instructions (WHAT)

Think of `config.yaml` as a recipe card.

### What it does NOT do
- ❌ Download data
- ❌ Create folders
- ❌ Validate anything
- ❌ Execute logic

### What it answers
- What is the data source?
- Where should the data be stored?
- Which options are configurable?

### What it declares
- `source_url`
- Directory paths
- File names
- `unzip_dir`
- Other configurable parameters

### Key Rule
> YAML never executes anything.  
> It only describes what should happen.


```yaml
artifact_root: artifacts
data_ingestion:
  root_dir: artifacts/data_ingestion
  source_url: https://github.com/Yonithe23/dataset/raw/refs/heads/master/winequality-data.zip
  local_data_file: artifacts/data_ingestion/data.zip
  unzip_dirs: artifact_root/data_ingestion
### Summary
- ✅ Declarative (instructions only)
- ❌ Not executable

### One-Line (Interview-Ready)
> `config.yaml` defines what the pipeline should do, not how or when it runs.


## 2. `paths.py` / `config_path.py` → Where to Find Instructions (**WHERE**)

This file answers **one focused question**:

> **Where are the project’s configuration instructions located?**

It serves as a **centralized place for all important file paths**, such as configuration and schema files.

### Example Code (`paths.py` / `config_path.py`)

```python
from pathlib import Path

CONFIG_FILE_PATH = Path("config/config.yaml")

## 2. `paths.py` → Where to Find Instructions (**WHERE**)

This file answers **one simple but critical question**:

> **Where are the project instructions located?**

It acts as a **central registry for all important paths**, such as:

- `CONFIG_FILE_PATH`
- `SCHEMA_FILE_PATH`
- `PARAMS_FILE_PATH`

---

### Why This Matters

Centralizing paths is a **production best practice**:

- 🚫 Avoids hard-coded strings scattered across the codebase  
- 📍 Creates a **single source of truth** for file locations  
- 🔁 Makes refactoring easy if the project structure changes  
- 🛡️ Reduces fragile code and long-term maintenance cost  

---

### Key Benefits

- ✅ Prevents “magic strings” everywhere  
- ✅ Simplifies future folder or file movement  
- ✅ Keeps the codebase **clean, stable, and scalable**  


### 3. utils/common.py — Short Summary

- Reusable helper functions shared across the project  
- Handles reading, validating, and parsing files (YAML, JSON, directories)  
- Converts configuration files into Python-friendly objects (`ConfigBox`)  
- Keeps pipelines clean, stateless, and testable  

👉 `common.py` is imported in **data ingestion** to reuse standardized logic and avoid duplication.  
📌 For more details on how configuration files are read and validated, **refer to `utils/common.py`**.


## 4️⃣ `entity/` → WHAT it means
### Structured Understanding / Contract Layer

At this stage, raw configuration becomes **structured, predictable, and self-documenting**.

Entities convert **loose configuration data** into **strict contracts**.

---

### 🧠 Core Idea

Entities **define rules, not behavior**.

They describe **what a valid configuration looks like**, but they **do not**:
- read files
- assign values
- execute logic

> Think of an entity as an **empty bottle**  
> 🧴 It defines the shape  
> ❌ It does not fill itself

---

### 📌 What Happens Here

When using entities, configuration becomes:

- ✅ **Typed**
- ✅ **Predictable**
- ✅ **Self-documenting**

Raw config values are later mapped **into these typed objects** by the `ConfigurationManager`.

---

### 📁 Location

```text
src/entity/config_entity.py


🎯 Purpose
- Define structured configuration objects
- Use data classes to describe expected fields and types
- Act as a contract between YAML and Python logic

💡 Why This Matters (Senior-Level)
Entities provide:
- Type safety
- IDE auto-completion
- Self-documenting code
- Early error detection
- Clean separation of concerns

This is a senior-level MLOps / production pattern, not tutorial code.

✅ What Entities DO
- Define structure
- Enforce data types
- Act as contracts
- Describe valid configuration shape

❌ What Entities DO NOT Do
- Read YAML files
- Assign configuration values
- Execute business logic
- Create directories
- Perform any I/O


In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [ ]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

## 5️⃣ Configuration Manager — What It Actually Does

**The Configuration Manager acts as a bridge layer between configuration files and runtime objects.**

It does **not** execute business logic.  
It prepares and validates everything **before execution begins**.

---

### A️ Reads & Validates YAML Configuration

- Uses `utils/common.py`
- Reads raw instructions from:
  - `config.yaml`
  - `params.yaml`
  - `schema.yaml`

**Validation responsibilities:**
- Ensures required keys exist
- Ensures correct data types
- Fails early if configuration is broken

📌 **Why this matters:**  
Early validation prevents runtime crashes and hard-to-debug failures.

---

### B️ Creates Required Directories (Infrastructure Setup)

- Creates directories if they do not exist

📌 **Why here?**  
Directory creation is **infrastructure setup**, not **business logic**.  
This keeps components clean and focused.

---

### C️ Maps YAML Values → Entity Objects

- Converts validated YAML values into **typed entity objects**
- Example:
  - `config.yaml` → `DataIngestionConfig`

📌 **Why this is critical:**  
Entities act as **contracts** between configuration and components, ensuring:
- Type safety
- Clean interfaces
- Predictable runtime behavior

---

### 🔑 One-Line Summary (Interview-Ready)

> **“The Configuration Manager validates configuration, prepares infrastructure, and converts raw YAML into typed runtime objects before execution begins.”**


In [ ]:
import os
import urllib.request as request
import logging
import zipfile

## 6️⃣ components → HOW the work is implemented (BUSINESS LOGIC)

This is the **missing but critical layer** in a production-grade pipeline.

### 🔧 What components do
- Implement **real business logic**
- Perform **actual work**, such as:
  - Downloading data
  - Unzipping files
  - Saving raw data to disk
- Use **typed config from `entity/`**
- Contain **NO orchestration logic**

### 🚫 What components do NOT do
- Do not read YAML files
- Do not manage execution order
- Do not trigger other pipeline stages

### 🧠 Key idea
**Components = workers**

They:
- Do **one job**
- Do it **well**
- Are **reusable**, **testable**, and **independent**

### 🎯 Interview-ready one-liner
> *“Components implement the core business logic of the pipeline. They act as workers that execute a single responsibility using structured configuration, without handling orchestration.”*


In [ ]:

class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config
    
    # Downloading the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logging.info(f"{filename} download! with following info: \n{headers}")
        else:
            logging.info(f"File already exists")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


### 7️⃣ `pipeline/` → DO the actual work (ORCHESTRATION)

**Purpose**
- Coordinates the execution of the pipeline
- Controls *when* and *in what order* things run

**What it does**
- Calls `components/` in sequence
- Manages execution flow
- Triggers the next stage of the pipeline

**What it does NOT do**
- ❌ No business logic
- ❌ No data processing implementation
- ❌ No configuration parsing

**Key Point (Interview-Ready)**
> **The pipeline orchestrates the workflow — it does not implement the logic.**  
> Logic lives in `components/`; the pipeline only decides *when* to run them.

**Mental Model**
- `components/` = *workers*
- `pipeline/` = *conductor* 🎼


In [ ]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

# General Description

## 🧠 Correct Mental Model (Very Important)

### 🧴 Entity = Empty Bottle
- Defines **shape, size, and label** (data structure)
- Contains **no data**
- Describes **what values should look like**, not the values themselves

---

### 🧰 Configuration Manager = Fills the Bottle
- Reads **YAML configuration files**
- Validates values against **schema**
- Creates required **directories**
- Assigns validated values to **entity objects**

➡️ Converts **raw YAML → typed Python entities**

---

### ⚙️ Component = Uses the Bottle
- Receives **filled entity objects**
- Consumes configuration values
- Executes the **actual business logic**
  - Data ingestion
  - Data transformation
  - Model training
  - Artifact storage

---
## 🏗️ Design (Code Structure – Static View)

- entity  
  ↓  
- Configuration Manager  
  ↓  
- components/  
  ↓  
- pipeline/




- `entity/` → defines structure
- `Configuration Manager` → prepares validated, ready-to-use objects
- `components/` → implements business logic
- `pipeline/` → orchestrates execution order

---

## ▶️ Execution (Runtime – Dynamic View)

```text
pipeline
↓
Configuration Manager
↓
Entity objects created (filled bottles)
↓
Components execute using entity values
```


---

## 🎯 One-Line Interview Summary

> **“Entities define structure, the Configuration Manager injects validated values, and components execute business logic—while the pipeline orchestrates the workflow.”**
